/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0239 - val_loss: 3.5241e-04
Epoch 2/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.0018 - val_loss: 2.6666e-04
Epoch 3/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 3.0356e-04 - val_loss: 8.0875e-05
Epoch 4/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 1.2613e-04 - val_loss: 6.5409e-05
Epoch 5/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0250e-04 - val_loss: 9.2432e-05
Epoch 6/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 9.2389e-05 - val_loss: 8.2069e-05
Epoch 7/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 8.7998e-05 - val_loss: 5.6852e-05
Epoch 8/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 8.3630e-05 - val_loss: 5.4199e-05
Epoch 9/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 8.7237e-05 - val_loss: 1.0855e-04
Epoch 10/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 7.7599e-05 - val_loss: 4.4704e-05
Epoch 11/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0247 - val_loss: 2.6377e-04
Epoch 2/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0018 - val_loss: 1.6543e-04
Epoch 3/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2.2933e-04 - val_loss: 1.1386e-04
Epoch 4/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.3355e-04 - val_loss: 9.3368e-05
Epoch 5/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.1312e-04 - val_loss: 5.6805e-05
Epoch 6/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.2049e-04 - val_loss: 8.0307e-05
Epoch 7/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0740e-04 - val_loss: 5.1435e-05
Epoch 8/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.2842e-04 - val_loss: 5.7858e-05
Epoch 9/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1.0417e-04 - val_loss: 4.7267e-05
Epoch 10/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 8.1228e-05 - val_loss: 7.4732e-05
Epoch 11/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0146 - val_loss: 4.4972e-04
Epoch 2/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.0010 - val_loss: 1.0418e-04
Epoch 3/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 1.4396e-04 - val_loss: 8.0947e-05
Epoch 4/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0905e-04 - val_loss: 1.0131e-04
Epoch 5/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1.1049e-04 - val_loss: 7.6018e-05
Epoch 6/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1.1416e-04 - val_loss: 7.5997e-05
Epoch 7/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 8.2435e-05 - val_loss: 6.2365e-05
Epoch 8/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 9.2744e-05 - val_loss: 6.1790e-05
Epoch 9/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 9.0387e-05 - val_loss: 7.9544e-05
Epoch 10/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 9.6673e-05 - val_loss: 5.5027e-05
Epoch 11/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0146 - val_loss: 4.5633e-04
Epoch 2/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.0014 - val_loss: 1.2660e-04
Epoch 3/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2.1075e-04 - val_loss: 1.0974e-04
Epoch 4/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 1.2559e-04 - val_loss: 7.0949e-05
Epoch 5/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.1684e-04 - val_loss: 6.2306e-05
Epoch 6/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 9.3936e-05 - val_loss: 7.3328e-05
Epoch 7/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 9.4227e-05 - val_loss: 5.9277e-05
Epoch 8/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.1182e-04 - val_loss: 5.6672e-05
Epoch 9/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 7.5678e-05 - val_loss: 6.7444e-05
Epoch 10/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 8.6709e-05 - val_loss: 4.6127e-05
Epoch 11/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0187 - val_loss: 2.3023e-04
Epoch 2/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0016 - val_loss: 1.6222e-04
Epoch 3/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2.8873e-04 - val_loss: 8.9456e-05
Epoch 4/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1.4667e-04 - val_loss: 1.1558e-04
Epoch 5/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0476e-04 - val_loss: 5.4727e-05
Epoch 6/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0355e-04 - val_loss: 9.3542e-05
Epoch 7/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 9.9812e-05 - val_loss: 8.2091e-05
Epoch 8/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 7.7684e-05 - val_loss: 4.9312e-05
Epoch 9/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 8.8036e-05 - val_loss: 6.1995e-05
Epoch 10/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 8.5127e-05 - val_loss: 1.0028e-04
Epoch 11/30
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_excel('/ElPaso (1).xlsx')

df.dropna(inplace=True)


df['Hour'] = pd.to_datetime(df['Time']).dt.hour
df['DayOfWeek'] = pd.to_datetime(df['Time']).dt.dayofweek
df['Month'] = pd.to_datetime(df['Time']).dt.month
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)


for lag in range(1, 25):
    df[f'Lag_{lag}'] = df['D'].shift(lag)


df['Temp_RollingMean_24'] = df['Temperature'].rolling(window=24).mean()
df['Temp_RollingStd_24'] = df['Temperature'].rolling(window=24).std()


df['Temp_Humidity_Interaction'] = df['Temperature'] * df['RelativeHumidity']


df.dropna(inplace=True)


features = df[['Temperature', 'DewPoint', 'RelativeHumidity', 'Precipitation', 'WindSpeed', 'Pressure',
               'Hour', 'DayOfWeek', 'Month', 'IsWeekend',
               'Lag_1', 'Lag_2', 'Lag_3', 'Lag_4', 'Lag_5', 'Lag_6', 'Lag_7', 'Lag_8', 'Lag_9', 'Lag_10',
               'Lag_11', 'Lag_12', 'Lag_13', 'Lag_14', 'Lag_15', 'Lag_16', 'Lag_17', 'Lag_18', 'Lag_19',
               'Lag_20', 'Lag_21', 'Lag_22', 'Lag_23', 'Lag_24',
               'Temp_RollingMean_24', 'Temp_RollingStd_24',
               'Temp_Humidity_Interaction']].values
energy_demand = df['D'].values.reshape(-1, 1)
dates = df['Time'].values


scaler_features = MinMaxScaler()
scaler_energy = MinMaxScaler()
features_normalized = scaler_features.fit_transform(features)
energy_demand_normalized = scaler_energy.fit_transform(energy_demand)

def create_sequences(data, target, dates, seq_length):
    sequences = []
    targets = []
    sequence_dates = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_seq = target[i+seq_length]
        date_seq = dates[i+seq_length]
        sequences.append(seq)
        targets.append(target_seq)
        sequence_dates.append(date_seq)
    return np.array(sequences), np.array(targets), np.array(sequence_dates)

seq_length = 24
X, y, dates_seq = create_sequences(features_normalized, energy_demand_normalized, dates, seq_length)

kf = KFold(n_splits=5, shuffle=False, random_state=None) #CHANGED from true to false, removed 42 random state

mse_scores = []
mae_scores = []
mape_scores = []

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

result_df = pd.DataFrame()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dates_test = dates_seq[test_index]

    model = Sequential([
        LSTM(50, activation='relu', input_shape=(seq_length, X.shape[2])),
        Dropout(0.2), #added dropout layer
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    #adjust patience value below
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )


    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1) #30

    predictions_normalized = model.predict(X_test)
    predictions = scaler_energy.inverse_transform(predictions_normalized)
    y_test_original = scaler_energy.inverse_transform(y_test)

    mse = mean_squared_error(y_test_original, predictions)
    mae = mean_absolute_error(y_test_original, predictions)
    mape = mean_absolute_percentage_error(y_test_original, predictions)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)

    fold_results = pd.DataFrame({
        'Date': dates_test,
        'Actual': y_test_original.flatten(),
        'Predicted': predictions.flatten()
    })
    result_df = pd.concat([result_df, fold_results], ignore_index=True)

result_df.to_csv('el paso_predictions.csv', index=False)

average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_mape = np.mean(mape_scores)
print(f"Average Mean Squared Error: {average_mse}")
print(f"Average Mean Absolute Error: {average_mae}")
print(f"Average Mean Absolute Percentage Error: {average_mape}")

plt.figure(figsize=(12, 6))
plt.plot(result_df['Actual'][:100], label='Actual')
plt.plot(result_df['Predicted'][:100], label='Predicted')
plt.title('Predicted vs Actual Energy Demand (Last Fold)')
plt.xlabel('Time')
plt.ylabel('Energy Demand')
plt.legend()
plt.show()